# Introduction #

### 1) Apply Pooling

In [ ]:
# YOUR CODE HERE
image_condense = ____

q_4.check()

In [ ]:
#%%RM_IF(PROD)%%
q_4.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_4.hint()
#_COMMENT_IF(PROD)_
q_4.solution()

Run the next cell to see what maximum pooling did to the feature!

In [ ]:
plt.imshow(
    # Reformat for plotting
    tf.squeeze(image_detect)
)
plt.axis('off')
plt.show();

### 2) Explore Invariance

This next code cell will randomly apply a small shift to the circle and then apply maximum pooling several times. Run this cell a few times and make note of the feature produced at the end.

If you were wanting to teach a classifier to recognize circles, would it be helpful to include all of these "shifted" circles in your dataset?

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_4.hint()
#_COMMENT_IF(PROD)_
q_4.solution()

In Lesson 6, we'll explore this idea more when we learn about **Data Augmentation**.

# Conclusion #